In [1]:
import pandas as pd
import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

# 전체 컬럼, 전체 행, 전체 숫자 출력
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)

# # plot 출력
# %matplotlib inline

# # plot 한글출력
# import matplotlib.font_manager as fm

# for font in fm.fontManager.ttflist:
#     if font.name in ['AppleGothic', 'NanumGothic']:
#         plt.rcParams['font.family'] = font.name
#     break

# warning message 제거
import warnings
warnings.filterwarnings('ignore')

In [4]:
boxoffice = pd.read_csv('films_updated.csv')
print(boxoffice.shape)
boxoffice.sample()

(686, 38)


,영화명,영화코드,개봉일,대표국적,국적,장르,등급,영화형태,영화구분,제작사,배급사,상영횟수,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,F-rated,감독,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모
505,박화영,20171681,2018-07-19,한국,한국,드라마,청소년관람불가,장편,독립/예술영화,명필름랩,(주)리틀빅픽쳐스,436,16,45361700.000000,5463,37613000,4331,1,이환,이환,10058251,남자,감독,어른들은 몰라요|박화영|캐릭터|너와 나의 21세기|똥파리|황산벌|집|창|지랄|실종자...,이환,이환,이환,10058251,남자,감독,어른들은 몰라요|박화영|캐릭터|너와 나의 21세기|똥파리|황산벌|집|창|지랄|실종자...,"김가희,강민아,이재균,이유미,김도완,하윤경,오하늬,차순배,한별,우지현,강태영,김나연...",김가희,혹금성(박석영),20273689,여자,배우,박화영|꿈의 제인|수상한 그녀|처녀비행|점프샷|집


In [5]:
k_movie_list = boxoffice['영화명']

len(k_movie_list)

686

In [33]:
df = pd.read_csv('Theaters_merged_files/CGV%20판교.csv')
print(df.shape)
df.sample()

(10743, 14)


,상영일자,상영관,1회,2회,3회,4회,5회,6회,7회,8회,9회,10회,11회,좌석수
4638,2017-06-11,07관,"09:45, 노무현입니다(디지털), 7,000원","12:00, 원더 우먼(디지털), 12,000원","14:45, 원더 우먼(디지털), 12,000원","17:35, 원더 우먼(디지털), 12,000원","20:25, 원더 우먼(디지털), 12,000원","23:10, 원더 우먼(디지털), 12,000원",NaN,NaN,NaN,NaN,NaN,184


In [13]:
theaters = pd.read_csv('Theaters_crawled_files/theaters_191128.csv')
theater_list = theaters['영화상영관명']

In [ ]:
timetable_preprocessing()

In [15]:
theater_list.sample()

509    롯데시네마 성남신흥
Name: 영화상영관명, dtype: object

In [37]:
data = pd.DataFrame()
def timetable_preprocessing(name):
    file_name = name.replace(' ', '%20').replace('(', '%28').replace(')', '%29')
    df = pd.read_csv(f'Theaters_merged_files/{file_name}.csv')
#     상영시각 따로 떼서 어느 그룹에 들어가는지 확인 [날짜, 상영관, 상영시각, 영화명, 가격] + [상영시각 등급]
# name = 'CGV 판교'
    timetable_spread = list() # 리스트

    df_dict = df.T.to_dict()

    for i in df_dict: # row 를 dict 로, 컬럼도 추려서

        row = df_dict[i]
        # 컬럼명에서 따오기
        for n in range(1, 12):
            if pd.isna(row[f'{n}회']):
                continue

            detail = row[f'{n}회'].split(', ')

            # 튜플이나 딕셔너리
            data = dict()
            data['상영관'] = row['상영관']
            data['상영일자'] = row['상영일자']
            data['상영시각'] = detail[0]
            data['영화명'] = detail[1]
            data['좌석수'] = row['좌석수']

            if len(detail) > 2:
                data['가격'] = detail[-1]

            # 리스트에 append
            timetable_spread.append(data)

    print(f'*** {name} 테이블 변환 완료 ***')

    # # 한국영화만 가져오기
    # df.merge(k_movie_list)

    timetable_spread = pd.DataFrame(timetable_spread)
    timetable_spread['영화관'] = name
    
#     return timetable_spread
    timetable_spread.to_csv(f'Theaters_merged_files/{name}_timetable_spread.csv', index=False)

In [38]:
# 위에서 지정해준 인덱스 숫자 n과 m을 사용해 code['영화상영관명'][n:m] 으로 변경
for name in theater_list[:300]:
    try:
        print(name, '처리 시도 중...')
#         data = 
        timetable_preprocessing(name)
        print(name, '완료')
    except ValueError as e:
        print('[Error]', e)
#         data = pd.concat([data, timetable_spread]) 
print('*** 완료!!! ***')

롯데시네마 원주무실 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xb7 in position 22: invalid start byte
CGV 울산신천 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xbf in position 28: invalid start byte
CGV 오산중앙 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xbf in position 28: invalid start byte
롯데시네마 대전관저 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xb7 in position 22: invalid start byte
CGV 동수원 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xb5 in position 28: invalid start byte
CGV 명동역 씨네라이브러리 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xb8 in position 28: invalid start byte
CGV 정관 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xc1 in position 28: invalid start byte
CGV 화성봉담 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xbc in position 30: invalid start byte
대한극장(서울) 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xb4 in position 22: invalid start byte
메가박스 성수점 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xb8 in position 22: invalid sta

*** CGV 판교 테이블 변환 완료 ***
CGV 판교 완료
CGV 대전 처리 시도 중...
*** CGV 대전 테이블 변환 완료 ***
CGV 대전 완료
롯데시네마 안동 처리 시도 중...


KeyError: '11회'

In [28]:
pd.read_csv('Theaters_merged_files/명보시네마.csv')

,상영일자,상영관,1회,2회,3회,4회,5회,6회,7회,8회,좌석수
0,2017-12-26,01관,"22:00, 킹콩(필름)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,94
1,2017-12-26,02관,"22:00, 킹콩(필름)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,95
2,2017-12-26,03관,"22:00, 킹콩(필름)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,154
3,2017-12-27,01관,"10:20, 1987(디지털)","12:45, 1987(디지털)","16:50, 1987(디지털), 8,000원","19:10, 1987(디지털), 8,000원","21:35, 1987(디지털), 8,000원",NaN,NaN,NaN,94
4,2017-12-27,02관,"13:20, 강철비(디지털)","16:00, 강철비(디지털), 8,000원","18:40, 강철비(디지털), 8,000원",NaN,NaN,NaN,NaN,NaN,95
5,2017-12-27,03관,"09:50, 신과함께-죄와 벌(디지털)","12:25, 신과함께-죄와 벌(디지털)","15:05, 신과함께-죄와 벌(디지털), 8,000원","17:45, 신과함께-죄와 벌(디지털), 8,000원","20:30, 신과함께-죄와 벌(디지털), 8,000원",NaN,NaN,NaN,154
6,2017-12-28,01관,"10:20, 1987(디지털), 6,000원","12:45, 1987(디지털), 8,000원","15:15, 뽀로로 극장판 공룡섬 대모험(디지털), 8,000원","16:50, 1987(디지털), 8,000원","19:10, 1987(디지털), 8,000원","21:35, 1987(디지털), 8,000원",NaN,NaN,94
7,2017-12-28,02관,"09:50, 신과함께-죄와 벌(디지털)","12:25, 신과함께-죄와 벌(디지털), 8,000원","15:05, 신과함께-죄와 벌(디지털), 8,000원","17:45, 신과함께-죄와 벌(디지털), 8,000원","20:30, 신과함께-죄와 벌(디지털), 8,000원",NaN,NaN,NaN,95
8,2017-12-28,03관,"09:50, 신과함께-죄와 벌(디지털), 6,000원",NaN,NaN,NaN,NaN,NaN,NaN,NaN,154
9,2017-12-29,01관,"09:30, 1987(디지털)","11:50, 1987(디지털), 8,000원","14:20, 강철비(디지털), 8,000원","16:50, 신과함께-죄와 벌(디지털), 8,000원","19:30, 1987(디지털), 8,000원","21:55, 1987(디지털), 8,000원",NaN,NaN,94


In [ ]:
# concat 후에 할 작업
# 영화명 뒤에 붙어있는 (디지털), (IMAX) 등 따로 떼기
    timetable_spread['상영분류'] = timetable_spread['영화명']
    timetable_spread['상영분류'] = timetable_spread['상영분류'].apply(lambda x: x.split('(')[1].replace(')', '') if len(x.split('(')) > 1 else np.nan)
    timetable_spread['영화명'] = timetable_spread['영화명'].apply(lambda x: x.split('(')[0])

    # 상영일자에서 요일 정보 컬럼 만들기 -> 주중(월화수목) 주말(금토일) 로 나누기
    timetable_spread['상영일자'] = pd.to_datetime(timetable_spread['상영일자'])
    timetable_spread['year'] = timetable_spread['상영일자'].dt.year
    timetable_spread['month'] = timetable_spread['상영일자'].dt.month
    timetable_spread['day'] = timetable_spread['상영일자'].dt.day
    timetable_spread['dayofweek'] = timetable_spread['상영일자'].dt.dayofweek

    timetable_spread['주말'] = 0
    # timetable_spread.loc[timetable_spread['dayofweek'] == 4, '주말'] = 1 # 금요일도 주중이니까 시간대 분류를 위해
    timetable_spread.loc[timetable_spread['dayofweek'] == 5, '주말'] = 1 
    timetable_spread.loc[timetable_spread['dayofweek'] == 6, '주말'] = 1 

    timetable_spread[['상영일자', 'year', 'month', 'day', 'dayofweek', '주말']]
    
    # 상영시각 등급
    timetable_spread.loc[timetable_spread['상영시각'] > '24:00', '상영시각등급'] = '나이트'
    timetable_spread.loc[timetable_spread['상영시각'] <= '24:00', '상영시각등급'] = '문라이트'
    timetable_spread.loc[timetable_spread['상영시각'] <= '22:00', '상영시각등급'] = '프라임'
    timetable_spread.loc[timetable_spread['상영시각'] <= '16:00', '상영시각등급'] = '데이라이트'
    timetable_spread.loc[timetable_spread['상영시각'] <= '13:00', '상영시각등급'] = '브런치'
    timetable_spread.loc[timetable_spread['상영시각'] <= '10:00', '상영시각등급'] = '모닝'
